In [1]:
import Data.Bits
import Data.Char
import Numeric
import qualified Data.Vector.Unboxed as V
import qualified Data.Set as S

In [2]:
twist :: Int -> Int -> [Int] -> [Int]
twist i len ks =
  let (before, after) = splitAt i ks
      unwrapped = after ++ before
      (a, b) = splitAt len unwrapped
      a' = reverse a
      unwrapped' = a' ++ b
  in uncurry (flip (++)) $ splitAt (length after) unwrapped'

In [3]:
run :: [Int] -> [Int] -> [Int]
run ks lens = run' ks lens 0 0
  where run' ks [] pos skip = ks
        run' ks (len:lens) pos skip =
          run' (twist pos len ks) lens (mod (pos + skip + len) (length ks)) (skip + 1)

In [4]:
hash :: String -> [Int]
hash s =
  let lens = map ord s ++ [17, 31, 73, 47, 23]
      sparse = run [0..255] $ concat $ replicate 64 lens
      in getDense sparse
      
  where getDense :: [Int] -> [Int]
        getDense [] = []
        getDense s = let (a,b) = splitAt 16 s in (foldl xor 0 a):(getDense b)

In [5]:
sum $ popCount <$> concat [hash $ "jzgqcdpd-" ++ show i | i <- [0..127]]

8074

In [6]:
explode :: Int -> [Int]
explode i = reverse [if testBit i b then 1 else 0 | b <- [ 0..7]]

In [7]:
flat = concatMap explode $ concat [hash $ "jzgqcdpd-" ++ show i | i <- [0..127]]

In [8]:
vec = V.fromList [if a /= 0 then b else 0 | (a, b) <- zip flat $ scanl (+) 1 flat]

In [9]:
toIndex :: Int -> Int -> Int
toIndex row col = (col * 128 + row)

fromIndex :: Int -> (Int, Int)
fromIndex i = quotRem i 128

In [10]:
cell :: V.Vector Int -> Int -> Int -> Int
cell v row col = if ((row < 0) || (row > 127) || (col < 0) || (col > 127)) then 0
                 else v V.! (toIndex row col)

In [11]:
maxNeighbor :: V.Vector Int -> Int -> Int -> Int
maxNeighbor v row col = maximum [cell v (row + i) (col + j) | (i,j) <- [(1,1), (-1,1), (1,-1), (-1,-1)]]

In [12]:
updateList v = [(toIndex row col, maxNeighbor v row col) | row <- [0..127], col <- [0..127],
                maxNeighbor v row col /= 0]

In [13]:
update :: V.Vector Int -> V.Vector Int
update v = let l = updateList v in
  if null l then v else update $ v V.// (updateList v)

In [ ]:
S.size $ S.fromList $ V.toList $ update vec